In [1]:
import mbuild as mb 

import numpy as np 


from pathlib import Path
import sys
directory = Path('/Users/kieran/surface_coatings')
sys.path.append(str(directory))

# from surface_coatings import surface_coatings 
from surface_coatings.surfaces import SilicaInterfaceCarve, SiliconInterface, SilicaInterface
from surface_coatings.chains import SilanePolymer, VBCPolymer

from surface_coatings import Monolayer
from mbuild.lib.atoms import H
from surface_coatings.chains import SilanePolymer
from surface_coatings.monomers.mpc import MPC
from mbuild.lib.moieties import Silane

from surface_coatings import SolvatedDualMonolayer, DualMonolayer
import foyer 
from foyer import Forcefield

/Users/kieran/mbuild/mbuild/utils/decorators.py:134: Warning: freud_generate_bonds is an experimental feature and is not subject to follow standard deprecation cycles. Use at your own risk! 
  warn(


In [2]:
monomer = SilanePolymer(n=1)
# display(monomer.visualize(backend='nglview',show_ports=True))
comps = list(monomer.particles())
monomer.remove([comps[0],comps[2],comps[3],comps[36]])
print(monomer.all_ports())
first_h = H()
second_h = H()
monomer.add(first_h)
monomer.add(second_h)
mb.force_overlap(first_h, first_h['up'], monomer['down'])
mb.force_overlap(second_h, second_h['up'], monomer.available_ports()[0])
display(monomer.visualize(backend='nglview'))
# monomer.save('length_1_mpc_silane_monomer_h_capped.mol2')
# silane = mb.load('[SiH4]',smiles=True)
# comps = list(silane.particles())
# silane.remove([comps[1],comps[2]])
# # silane.add()



[<Port, anchor: 'C', labels: , id: 140368148961696>, <Port, anchor: 'C', labels: ['Polymer']['down'], id: 140368149038176>, <Port, anchor: 'Si', labels: ['Silane']['down'], id: 140368148993696>]


/Users/kieran/mbuild/mbuild/conversion.py:481: UserWarning: Using parmed reader. Bonds may be inferred from inter-particle distances and standard residue templates. Please check that the bonds in mb.Compound are accurate
  warn(


NGLWidget()

In [4]:
chain = mb.load('length_1_mpc_silane_monomer_h_capped.mol2')
oplsaa = Forcefield(forcefield_files="./utils/oplsaa_imodels.xml")
typed_chain = oplsaa.apply(chain, verbose=True,
                          assert_angle_params=False,
                          assert_dihedral_params=False)
typed_chain.save('length_1_mpc_silane_monomer_h_capped.top', overwrite=True)

/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/mdtraj/core/trajectory.py:438: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn('top= kwargs ignored since this file parser does not support it')
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:629: UserWarning: No force field version number found in force field XML file.
  warnings.warn(
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:641: UserWarning: No force field name found in force field XML file.
  warnings.warn(
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:653: UserWarning: No combining rule found in force field XML file.
  warnings.warn(
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/validator.py:165: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWa

Missing angle with ids (0, 1, 2) and types ['opls_142', 'opls_465', 'opls_467'].
Missing angle with ids (0, 33, 37) and types ['opls_142', 'opls_1004', 'opls_1002'].
Missing angle with ids (33, 37, 42) and types ['opls_1004', 'opls_1002', 'opls_9999'].
Missing angle with ids (38, 37, 42) and types ['opls_1005', 'opls_1002', 'opls_9999'].
Missing angle with ids (40, 37, 42) and types ['opls_1005', 'opls_1002', 'opls_9999'].
Missing dihedral with ids (0, 1, 2, 3) and types ['opls_142', 'opls_465', 'opls_467', 'opls_182'].
Missing dihedral with ids (1, 0, 33, 34) and types ['opls_465', 'opls_142', 'opls_1004', 'opls_140'].
Missing dihedral with ids (1, 0, 33, 35) and types ['opls_465', 'opls_142', 'opls_1004', 'opls_140'].
Missing dihedral with ids (2, 1, 0, 33) and types ['opls_467', 'opls_465', 'opls_142', 'opls_1004'].
Missing dihedral with ids (2, 1, 0, 43) and types ['opls_467', 'opls_465', 'opls_142', 'opls_144'].
Missing dihedral with ids (33, 0, 1, 36) and types ['opls_1004', 'opl

/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all angles. Total system angles: 77, Parameterized angles: 72
  warnings.warn(msg)
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all proper dihedrals. Total system dihedrals: 96, Parameterized dihedrals: 89. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers.
  warnings.warn(msg)
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 46, Parameterized impropers: 2. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)
/Users/kieran/opt/miniconda3/en

In [3]:
test = mb.load('silane_pmpc.xyz')
for part in test.particles():
    if "Si" in part.name:
        part.element = 'Si'
    else:
        part.element = part.name[0]
test2 = mb.load('test.pdb', backend='parmed')
display(test.visualize(backend='nglview'))
bonds = list(test2.bonds())
for b in bonds:
    if test2[0] in b:
        for part in b:
            if part.element.symbol == 'H':
                bond_to_remove = b
                break
# print(bond_to_remove)
test2.remove_bond(bond_to_remove)
display(test2.visualize(backend='nglview'))
# test2.save('silane_pmpc_moner_fixed.mol2',overwrite=True)

/Users/kieran/mbuild/mbuild/formats/xyz.py:68: UserWarning: No matching element found for Si2; the particle will be added to the compound without an element attribute.
  warn(
/Users/kieran/mbuild/mbuild/formats/xyz.py:68: UserWarning: No matching element found for OH; the particle will be added to the compound without an element attribute.
  warn(
/Users/kieran/mbuild/mbuild/formats/xyz.py:68: UserWarning: No matching element found for CH2; the particle will be added to the compound without an element attribute.
  warn(
/Users/kieran/mbuild/mbuild/formats/xyz.py:68: UserWarning: No matching element found for CM3; the particle will be added to the compound without an element attribute.
  warn(
/Users/kieran/mbuild/mbuild/formats/xyz.py:68: UserWarning: No matching element found for HC; the particle will be added to the compound without an element attribute.
  warn(
/Users/kieran/mbuild/mbuild/formats/xyz.py:68: UserWarning: No matching element found for CM1; the particle will be added 

NGLWidget()

NGLWidget()

In [9]:

surface = SilicaInterfaceCarve()
chain = SilanePolymer(n=2)

/Users/kieran/mbuild/mbuild/compound.py:755: UserWarning: The Compound you are adding has a box. The box of the parent compound will be used. Use inherit_box = True if you wish to replace the parent compound box with that of Compound being added.
  warn(
/Users/kieran/mbuild/mbuild/utils/decorators.py:134: Warning: freud_generate_bonds is an experimental feature and is not subject to follow standard deprecation cycles. Use at your own risk! 
  warn(


In [12]:
chain = SilanePolymer(n=2)
oplsaa = Forcefield(forcefield_files="./utils/oplsaa_imodels.xml")
typed_chain = oplsaa.apply(chain, verbose=True,
                          assert_angle_params=False,
                          assert_dihedral_params=False)
typed_chain.save('chain.top', overwrite=True)
display(typed_chain.visualize(backend='nglview'))
display(chain.visualize(backend='nglview'))

Missing angle with ids (42, 45, 46) and types ['opls_141', 'opls_465', 'opls_467'].
Missing dihedral with ids (1, 78, 42, 45) and types ['opls_139', 'opls_136', 'opls_141', 'opls_465'].
Missing dihedral with ids (4, 1, 78, 42) and types ['opls_465', 'opls_139', 'opls_136', 'opls_141'].
Missing dihedral with ids (41, 42, 45, 46) and types ['opls_135', 'opls_141', 'opls_465', 'opls_467'].
Missing dihedral with ids (41, 42, 45, 81) and types ['opls_135', 'opls_141', 'opls_465', 'opls_466'].
Missing dihedral with ids (42, 45, 46, 47) and types ['opls_141', 'opls_465', 'opls_467', 'opls_182'].
Missing dihedral with ids (43, 41, 42, 45) and types ['opls_140', 'opls_135', 'opls_141', 'opls_465'].
Missing dihedral with ids (44, 41, 42, 45) and types ['opls_140', 'opls_135', 'opls_141', 'opls_465'].
Missing dihedral with ids (45, 42, 41, 77) and types ['opls_465', 'opls_141', 'opls_135', 'opls_140'].
Missing dihedral with ids (45, 42, 78, 79) and types ['opls_465', 'opls_141', 'opls_136', 'opls

/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all angles. Total system angles: 208, Parameterized angles: 207
  warnings.warn(msg)
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all proper dihedrals. Total system dihedrals: 271, Parameterized dihedrals: 259. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers.
  warnings.warn(msg)
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 129, Parameterized impropers: 5. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)
/Users/kieran/opt/minicond

NGLWidget()

NGLWidget()

In [20]:
monolayer = Monolayer(surface=mb.clone(surface), 
                      chains=mb.clone(chain), 
                      n_chains=20, 
                      tile_x=1, 
                      tile_y=1)
dual_monolayer = DualMonolayer(top=mb.clone(monolayer),
                               bottom=mb.clone(monolayer),
                               separation=3,
                               shift=True)
solvated_dual_monolayer = SolvatedDualMonolayer(dual_monolayer=mb.clone(dual_monolayer))

/Users/kieran/surface_coatings/surface_coatings/monolayer.py:84: UserWarning: 
 No fractions provided. Assuming a single chain type.
  warn("\n No fractions provided. Assuming a single chain type.")
/Users/kieran/mbuild/mbuild/compound.py:768: UserWarning: After adding new Compound, Compound.box.lengths < Compound.boundingbox.lengths. There may be particles outside of the defined simulation box
  warn(
/Users/kieran/mbuild/mbuild/compound.py:1172: UserWarning: Compound.box.lengths < Compound.boundingbox.lengths. There may be particles outside of the defined simulation box.
  warn(


In [24]:
solvated_dual_monolayer.visualize('nglview')

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [28]:
oplsaa = Forcefield(forcefield_files="./utils/oplsaa_imodels.xml")
typed_monolayer = oplsaa.apply(chain, verbose=True,
                          assert_angle_params=False,
                          assert_dihedral_params=False)
typed_monolayer.save('solvated_dual_monolayer.top', overwrite=True)

/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:629: UserWarning: No force field version number found in force field XML file.
  warnings.warn(
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:641: UserWarning: No force field name found in force field XML file.
  warnings.warn(
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:653: UserWarning: No combining rule found in force field XML file.
  warnings.warn(
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/validator.py:165: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)


Missing angle with ids (1, 4, 5) and types ['opls_141', 'opls_465', 'opls_467'].
Missing dihedral with ids (0, 1, 4, 5) and types ['opls_135', 'opls_141', 'opls_465', 'opls_467'].
Missing dihedral with ids (0, 1, 4, 40) and types ['opls_135', 'opls_141', 'opls_465', 'opls_466'].
Missing dihedral with ids (1, 4, 5, 6) and types ['opls_141', 'opls_465', 'opls_467', 'opls_182'].
Missing dihedral with ids (1, 37, 62, 60) and types ['opls_141', 'opls_136', 'opls_139', 'opls_465'].
Missing dihedral with ids (2, 0, 1, 4) and types ['opls_140', 'opls_135', 'opls_141', 'opls_465'].
Missing dihedral with ids (3, 0, 1, 4) and types ['opls_140', 'opls_135', 'opls_141', 'opls_465'].
Missing dihedral with ids (4, 1, 0, 36) and types ['opls_465', 'opls_141', 'opls_135', 'opls_140'].
Missing dihedral with ids (4, 1, 37, 38) and types ['opls_465', 'opls_141', 'opls_136', 'opls_140'].
Missing dihedral with ids (4, 1, 37, 39) and types ['opls_465', 'opls_141', 'opls_136', 'opls_140'].
Missing dihedral wi

/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all angles. Total system angles: 130, Parameterized angles: 129
  warnings.warn(msg)
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all proper dihedrals. Total system dihedrals: 167, Parameterized dihedrals: 155. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers.
  warnings.warn(msg)
/Users/kieran/opt/miniconda3/envs/S_to_Si/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 80, Parameterized impropers: 4. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)
/Users/kieran/opt/miniconda